In [3]:
import os
import tarfile
import pdb
import csv
import io
import numpy as np
from statistics import stdev
import json
import orjson
import math
from multiprocessing import Process



all_compare_names = ['sameEv-sameSchema_sameEv-otherSchema',
                        'sameEv-sameSchema_otherEv-sameSchema',
                        'sameEv-sameSchema_otherEv-otherSchema',
                        'sameEv-otherSchema_otherEv-sameSchema',
                        'sameEv-otherSchema_otherEv-otherSchema',
                        'otherEv-sameSchema_otherEv-otherSchema']

# mark where each event starts and ends
event_1_start = 17
event_2_start = 23
event_3_start = 34
event_4_start = 50
event_5_start = 66

def run_jobs(input_dir, json_path, testing = False,  
            output_dir = "/scratch/network/rk1593/", 
            only_events2_3_and_4 = True,
            job_id_target = None, num_chunks = 31):
    """
    run a job which is a a particular set of searchlights that we want
    to turn into a tvalue vector for the purpose of clustering
    """
    # open the jobs info dict which is a dictionary containing a mapping
    # from job id (i.e. a number from 0 to 200) to the list of searchlights
    # to process in that job
    f = open(json_path,)
    jobs_info_dict = json.load(f)
    for job_id in range(jobs_info_dict["num_jobs_actual"]):
        # this is how we only process this one job of interest
        if job_id != job_id_target:
            continue
        print("job: ", job_id)
        this_job_searchlights = jobs_info_dict["job_id_to_searchlight_subset"][str(job_id)][:10]
        chunk = this_job_searchlights
        go_from_480searchlight_files_representing_fingerprintPlot_to_tvalue_vector(
                            chunk, 
                            testing, 
                            output_dir, 
                            only_events2_3_and_4,
                            input_dir,
                            )

# requires: searchlight_to_files_tuples for all searchlights, subset_list_of_searchlights are a list of the searchlights we want to include for this job
# outputs a csv file for each searchlight
def go_from_480searchlight_files_representing_fingerprintPlot_to_tvalue_vector(subset_list_of_searchlights, 
            testing = False, 
            output_dir = "", 
             only_events2_3_and_4 = True,
              in_dir = ""):
    os.chdir(in_dir)
    for counter_light, light_id in enumerate(subset_list_of_searchlights):
        tar_file_name = light_id + "_new.tar.gz"
        # check that we have a tar file for this searchlight
        if not os.path.exists(in_dir + tar_file_name):
            print("__________Light not on della yet!_________")
            continue
        tar = tarfile.open(tar_file_name)
        # get the 480 files and make sure we got the right number
        files_this_light = [x.name for x in tar.getmembers()]
        num_files_this_light = len(files_this_light)
        if num_files_this_light != 480:
            print("Error: searchlight " + light_id + " has " + str(num_files_this_light) + " files." )
            np.savetxt(output_dir +  "/searchlights_tval_example/" + light_id + "_NOT480.csv", np.array([1,2,3]), delimiter=",")
            return
        # don't reprocess a searchlight again if we already did create
        # tvalue vector for it in the past
        save_path = output_dir +  "/searchlights_tval_example/" + light_id + ".csv"
        if os.path.exists(save_path):
            print("Path Exists, Do Not Reprocess")
            continue
        template2_count = 0
        template3_count = 0
        template4_count = 0
        template_to_pid_to_cond_to_matrices = {} # create this dict for later usage when creating fingerprint plots
        template_to_pid_to_cond_to_lists = {}
        i = 0
        # step 1 is to get dict mapping the template (2,3 or 4) to pid (N = 40) to cond (condition) where condition is synonymous
        # with "other-Event-same-Schema" and so on
        for file_name in files_this_light:
            splitted_file_name = file_name.split("_")
            # get the participant id
            pid = splitted_file_name[1]
            cond = splitted_file_name[3] + "-" + splitted_file_name[4]
            template_id = int(file_name[-5])
            if file_name[-5] == "2":
                template2_count += 1 
            elif file_name[-5] == "3":
                template3_count += 1 
            elif file_name[-5] == "4":
                template4_count += 1 
            else:
                print("Error: file_name template retrieval error.")
                print(file_name)
                print(tar_file_name)
                return
            # change the directory to this in_dir to be able to open the tar file
            os.chdir(in_dir)
            i += 1
            in_text = tar.extractfile(file_name).read()
            csv_file = io.StringIO(in_text.decode('ascii'))
            random_replacer_for_nothing = str(3e200) # this is set to be something super big (greater than 1), so that 
            # when we replace all numbers outside [-1,1] with NaN then these lines where no data is recorded become NaN
            # cleaning_mean_start = time.time()
            csv_lines = [[y.replace(" ", "") for y in x] for x in csv.reader(csv_file)]
            if testing:
                csv_lines[0][0] = "-1000"
            # replace any spots where there is no data collected with the random replacer (more explained above why I did this)
            csv_lines = [[y if y != "" else random_replacer_for_nothing for y in x] for x in csv_lines]
            try:
                new_arr = np.array(csv_lines).astype("float")
                # replace the empty due to the random replacer and other anomaly large numbers outside of [-1,1] (the range for correlation) found in raw data
                new_arr[abs(new_arr) > 1] = np.nan
                # now we edit the new_arr so that 0-indexed row 9, columns 47 through 73 become nan for subject 102
                if pid == "sub-102": # sub-102 had some issues in this area of the FMRI
                    new_arr[9,47:74] = np.nan
            except ValueError:
                pdb.set_trace()
            # if in testing mode we need to crop out the first column and the first row
            # since these files were different 
            if testing:
                mean_list = np.nanmean(new_arr[1:13, 1:75], axis = 0).tolist()
                new_arr = new_arr[1:13, 1:75].tolist()
            else:
                mean_list = np.nanmean(new_arr, axis = 0).tolist()
                new_arr = new_arr.tolist()
            # error check if the mean list has an nan in it which means that all weddings in one tr had nan
            if np.count_nonzero(np.isnan(mean_list)) != 0:
                print("Error: mean list has np.nan")
                # if isnan has all Falses and so everything is 0
                return 
            if str(template_id) not in template_to_pid_to_cond_to_matrices:
                template_to_pid_to_cond_to_matrices[str(template_id)] = {}
                template_to_pid_to_cond_to_lists[template_id] = {}
            if pid not in template_to_pid_to_cond_to_matrices[str(template_id)]:
                template_to_pid_to_cond_to_matrices[str(template_id)][pid] = {}
                template_to_pid_to_cond_to_lists[template_id][pid] = {}
            template_to_pid_to_cond_to_matrices[str(template_id)][pid][cond] = new_arr
            template_to_pid_to_cond_to_lists[template_id][pid][cond] = mean_list
        with open(output_dir + "searchlights_matrices_orjson_example/" + light_id, "wb") as f:
            f.write(orjson.dumps(template_to_pid_to_cond_to_matrices))
        # check that we got even number for each template for error check
        if template2_count != 160 or template3_count != 160 or template4_count != 160:
            print("Error!")
            print("tar_file_name: ", tar_file_name)
            print("template2_count: ", template2_count)
            print("template3_count: ", template3_count)
            print("template4_count: ", template4_count)
            return

        # step 2 is to get the dict mapping the template to the comparison
        # where there are 6 differen comparisons between the 4 conditions, to pid to 
        # a list of differences between one condition and another in the comparison
        template_to_compare_to_pid_to_tr_diffs = {}
        for template_id in template_to_pid_to_cond_to_lists:
            if template_id not in template_to_compare_to_pid_to_tr_diffs:
                template_to_compare_to_pid_to_tr_diffs[template_id] = {}
            for pid in template_to_pid_to_cond_to_lists[template_id]:
                for comparison_name in all_compare_names:
                    compare1_name,compare2_name = comparison_name.split("_")    
                    if comparison_name not in template_to_compare_to_pid_to_tr_diffs[template_id]:
                        template_to_compare_to_pid_to_tr_diffs[template_id][comparison_name] = {}
                    compare1_list =  template_to_pid_to_cond_to_lists[template_id][pid][compare1_name]
                    compare2_list = template_to_pid_to_cond_to_lists[template_id][pid][compare2_name]
                    if len(compare1_list) != len(compare2_list):
                        print("Error: for the same template and light, two paths have different number of tr's")
                        return
                    tr_differences = [(compare1_list[i] - compare2_list[i]) for i in range(len(compare1_list))]
                    if only_events2_3_and_4:
                        tr_differences = tr_differences[event_2_start:event_5_start]
                    template_to_compare_to_pid_to_tr_diffs[template_id][comparison_name][pid] = tr_differences

        # step 3: is to get the across subject tvalue for each comparison
        # at each TR for each template and comparison
        template_to_compare_to_trTstats = {}
        for template_id in template_to_compare_to_pid_to_tr_diffs:
            for compare_name in template_to_compare_to_pid_to_tr_diffs[template_id]:
                # check that all participants have the same length of tr_diffs
                # while also getting list of diffs at each tr
                length_tr_list = []
                tr_num_to_list_of_diffs = {}
                for pid in template_to_compare_to_pid_to_tr_diffs[template_id][compare_name]:
                    tr_diffs = template_to_compare_to_pid_to_tr_diffs[template_id][compare_name][pid]
                    for index,diff in enumerate(tr_diffs):
                        if index not in tr_num_to_list_of_diffs:
                            tr_num_to_list_of_diffs[index] = []
                        tr_num_to_list_of_diffs[index].append(diff)
                    length_tr_list.append(len(tr_diffs))
                length_tr_list = np.array(length_tr_list)
                if not np.all(length_tr_list[0] == length_tr_list):
                    pdb.set_trace()
                    print("Error: not all the same length_tr_lists")
                    return
                # now go through each tr index, and get a ttest
                # make sure that we have 40 participants in each tr num
                # and make sure we don't have nan
                for tr_num in tr_num_to_list_of_diffs:
                    if len(tr_num_to_list_of_diffs[tr_num]) != 40 or np.isnan(sum(tr_num_to_list_of_diffs[tr_num])):
                        print(tr_num_to_list_of_diffs[tr_num])
                        pdb.set_trace()
                        print("Error: missing participant or nan")
                        return
               
                tr_Tstats = [get_t_stat_of_list(tr_num_to_list_of_diffs[tr_num]) for tr_num in range(0,len(tr_num_to_list_of_diffs.keys()))]
                if template_id not in template_to_compare_to_trTstats:
                    template_to_compare_to_trTstats[template_id] = {}
                template_to_compare_to_trTstats[template_id][compare_name] = tr_Tstats
                # for each searchlight get a feature list
        # step 4: compile those tstats into one vector which is our final
        # tvalue vector representation that we will cluster
        features = []
        for template_id in [2,3,4]:
            for compare_name in all_compare_names:
                for tr_stat in template_to_compare_to_trTstats[template_id][compare_name]:
                    features.append(tr_stat)
        print("len(features): ", len(features))
        # output it!
        features_np = np.array(features)
        np.savetxt(save_path, features_np, delimiter=",")

def divide_chunks(l, chunk_size):
    """
    take in a list and chunk size and cut this list up into chunks
    """
    # looping till length l
    for i in range(0, len(l), chunk_size): 
        yield l[i:i + chunk_size]

def get_t_stat_of_list(list):
    """
    take in a list of size 40 for each particpant for a particular tr and comparison
    and output the tvalue
    """
    n = len(list)
    mean = (sum(list) / n)
    sd = stdev(list)
    sem = (sd / math.sqrt(n))
    t_stat = (mean / sem)
    return t_stat

# DRIVER #
bash_it = True
input_dir = "/scratch/gpfs/rk1593/tar_by_searchlight/tar_by_searchlight/" # here we have stored a list of 480 files in a tar file for each searchlight
output_dir = "/scratch/gpfs/rk1593/clustering_output/"  # output dhere on della
json_file_name = "jobs_info_dict_manual_jupyter_without_tuples.json"
testing = False
num_chunks = 31
only_events2_3_and_4 = True # we only want to look at event 2, 3 and 4
job_id_in = 0
print("job_id_in: ", job_id_in)
if bash_it:
    run_jobs(input_dir, json_path = output_dir + json_file_name, 
                testing = testing, 
            output_dir = output_dir,  only_events2_3_and_4 = only_events2_3_and_4,
            job_id_target= job_id_in,
            num_chunks= num_chunks)


job_id_in:  0
job:  0
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774
len(features):  774


# K-Means clustering

In [2]:

import pickle
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans


def kmeans_clustering(dir, K, X, searchlight_list):
    # read in the master_X_list and master_searchlight so that we have the cluster assignments
    # for the searchlights without having to output so much data each time
    # by outputting the master_X_list to the output csv multiple times
    # which would be a waste of della's memory
    print("K_clusters: ", K)
    kmeans = KMeans(n_clusters=K)
    cluster_labels = kmeans.fit_predict(X)
    # add 1 so that we have no label with zero
    cluster_labels += 1
    # create csv with the cluster assignment for this number of clusters
    # create the output df by giving cluster labels
    this_cost = kmeans.inertia_
    cost_list = [this_cost for i in range(len(searchlight_list))]
    output_dict = {"cluster_assignment": cluster_labels.tolist(),
                "searchlight": searchlight_list,
                "cost":cost_list
                }
    df = pd.DataFrame(output_dict)
    df.to_csv(dir + "kmeans_" + str(K) + "clusters_tval.csv")
    # It is important to use binary access
    with open(dir + "kmeans_" + str(K) + "clusters_tval.pickle", 'wb') as f:
        pickle.dump(kmeans, f)
        
        import numpy as np
import os

dir = "/scratch/gpfs/rk1593/clustering_output/"
X_list = []
searchlight_list = []
for index,file in enumerate(os.scandir(dir + "searchlights_tval_example/")):
    file_name = file.name
    if ".csv" in file_name:
        if "NOT480" in file_name:
            print("Error: NOT480 ", file_name)
        if index % 10000 == 0:
            print(index)
        light_id = file_name.split(".")[0]
        features = np.genfromtxt(dir + "searchlights_tval_example/" + file_name, delimiter = ",").tolist()
        searchlight_list += [light_id]
        X_list += [features]

print("extracted for /searchlights/ directory")
X = np.vstack(X_list)
dir = "/scratch/gpfs/rk1593/clustering_output/kmeans_assignments_tval_example/"
K = 5
kmeans_clustering(dir = dir, X = X, K = K, searchlight_list = searchlight_list)

0
extracted for /searchlights/ directory
K_clusters:  5


# P-values

In [ ]:
import json
import math
from scipy import stats
import msgpack
import orjson
from multiprocessing import Process
import numpy as np
import os


# this is to do the applied-to stuff2
# note that matching and applied-to are synonyms in my language
matching_id_labels_each_tr = []
for i in range(0,17):
    matching_id_labels_each_tr.append(0)
for i in range(17,23):
    matching_id_labels_each_tr.append(1)
for i in range(23,34):
    matching_id_labels_each_tr.append(2)
for i in range(34,50):
    matching_id_labels_each_tr.append(3)
for i in range(50,66):
    matching_id_labels_each_tr.append(4)
for i in range(66,74):
    matching_id_labels_each_tr.append(5)
matching_id_labels_each_tr = np.array(matching_id_labels_each_tr)


apply_id_to_it = {2: matching_id_labels_each_tr == 2,
                  3: matching_id_labels_each_tr == 3,
                  4: matching_id_labels_each_tr == 4}

path_names = ['sameEv-sameSchema', 'sameEv-otherSchema', 
                            'otherEv-sameSchema', 'otherEv-otherSchema']

path_abbrev_dict = {"sEsS": 'sameEv-sameSchema',
                    "sEoS": 'sameEv-otherSchema',
                    "oEsS": 'otherEv-sameSchema',
                    "oEoS": 'otherEv-otherSchema'}

roi_to_focus = {"schema": ["2_2", "3_2", "4_2", "2_3", "3_3", "4_3", "2_4", "3_4", "4_4"],
                "path": ["2_2", "3_2", "4_2", "2_3", "3_3", "4_3", "2_4", "3_4", "4_4"],
                "rotated": ["2_3", "3_2"],
                "perception": ["2_2","3_3","4_4"]}



def get_perception_value_each_tr(path_to_trs):
    return (path_to_trs['sameEv-sameSchema'] + path_to_trs['sameEv-otherSchema']) \
           - (path_to_trs['otherEv-sameSchema'] + path_to_trs['otherEv-otherSchema'])
   

# 2_3 and 3_2
def get_rotated_value_each_tr(path_to_trs, focus):
    if focus == "2_3":
        return (path_to_trs['otherEv-sameSchema'] + path_to_trs['otherEv-otherSchema']) \
                - (path_to_trs['sameEv-sameSchema'] + path_to_trs['sameEv-otherSchema'])
    elif focus == "3_2":
        return (path_to_trs['otherEv-sameSchema'] + path_to_trs['sameEv-otherSchema']) \
                - (path_to_trs['sameEv-sameSchema'] + path_to_trs['otherEv-otherSchema'])

def get_path_value_each_tr(path_to_trs):
    return path_to_trs['sameEv-sameSchema'] - path_to_trs['otherEv-sameSchema']

def get_schema_value_each_tr(path_to_trs):
    return (path_to_trs['sameEv-sameSchema'] + path_to_trs['otherEv-sameSchema']) \
            - (path_to_trs['sameEv-otherSchema'] + path_to_trs['otherEv-otherSchema'])

# roi to event of the form x_y where x is the template event and y is the applied-to event
def get_roi_to_focus_to_measure(pid, wedding, template_to_pid_to_cond_to_matrices, roi_in):
    # cond for condition and path are synonyms
    focus_to_measure = {}
    this_measures_list = []
    for focus in roi_to_focus[roi_in]:
        template_id = focus[0]
        appliedto_id = focus[2]
        apply_it = apply_id_to_it[int(appliedto_id)]
        path_to_trs = {}
        for path in path_names:
            # get the trs for this wedding, and in the correct appliedto id
            new_trs = np.array(template_to_pid_to_cond_to_matrices[template_id][pid][path], dtype = float)[wedding,:][apply_it]
            path_to_trs[path] = new_trs
        if roi_in == "schema":
            new_measure = get_schema_value_each_tr(path_to_trs).tolist()
        elif roi_in == "perception":
            new_measure = get_perception_value_each_tr(path_to_trs).tolist()
        elif roi_in == "rotated":
            new_measure = get_rotated_value_each_tr(path_to_trs, focus).tolist()
        elif roi_in == "path":
            new_measure = get_path_value_each_tr(path_to_trs).tolist()
        else:
            print("Error: roi invalid")
            return
        focus_to_measure[focus] = new_measure
        [this_measures_list.append(x) for x in new_measure]
    return focus_to_measure, this_measures_list
            

def get_template_to_pid_to_cond_to_matrix_msgpack(light_id, in_dir = "/scratch/gpfs/rk1593/clustering_output/"):
    template_to_pid_to_cond_to_matrices = {}
    with open(in_dir + "searchlights_matrices_msgpack/" + light_id, "rb") as json_file:
        template_to_pid_to_cond_to_matrices = msgpack.load(json_file, strict_map_key = False) 
    return template_to_pid_to_cond_to_matrices

def get_template_to_pid_to_cond_to_matrix(light_id, in_dir = "/scratch/gpfs/rk1593/clustering_output/"):
    template_to_pid_to_cond_to_matrices = {}
    with open(in_dir + "searchlights_matrices_orjson/" + light_id, "rb") as f:
        template_to_pid_to_cond_to_matrices = orjson.loads(f.read())
    return template_to_pid_to_cond_to_matrices

def get_roi_to_pvalue(pid_to_roi_to_measure):
    roi_to_pvalue = {}
    for roi in roi_to_focus.keys():
        across_pids = []
        for pid in pid_to_roi_to_measure:
            across_pids.append(pid_to_roi_to_measure[pid][roi])
        tstat,pval = stats.ttest_1samp(across_pids, popmean= 0, alternative = "greater")
        roi_to_pvalue[roi] = float(pval)
    return roi_to_pvalue

def get_event_to_measure(focus_to_measure, roi):
    event_list = [2,3,4] if roi != "rotated" else [2,3]
    event_to_measure = {}
    for event in event_list:
        if roi == "schema" or roi == "path":
            avg_measure = np.nanmean(np.hstack((focus_to_measure["2_" + str(event)],
                        focus_to_measure["3_" + str(event)],
                        focus_to_measure["4_" + str(event)])))
        elif roi == "perception":
            avg_measure = np.nanmean(focus_to_measure[str(event) + "_" + str(event)])
        elif roi == "rotated":
            if event == 3:
                avg_measure = np.nanmean(focus_to_measure["2_3"])
            elif event == 2:
                avg_measure = np.nanmean(focus_to_measure["3_2"])
        event_to_measure[str(event)] = float(avg_measure)
    return event_to_measure

def process_chunk(light_list, output_dir):
    for light_id in light_list:
        template_to_pid_to_cond_to_matrices = get_template_to_pid_to_cond_to_matrix(light_id)
        pid_to_roi_to_measure = {}
        pid_to_roi_to_wedding_to_event_to_measure = {}
        for pid in template_to_pid_to_cond_to_matrices["2"]:
            if pid not in pid_to_roi_to_measure:
                pid_to_roi_to_measure[pid] = {}
                pid_to_roi_to_wedding_to_event_to_measure[pid] = {}
            for roi in roi_to_focus.keys():
                if roi not in pid_to_roi_to_wedding_to_event_to_measure[pid]:
                    pid_to_roi_to_wedding_to_event_to_measure[pid][roi] = {}
                measures_list = []
                for wedding in range(12):
                    # focus
                    focus_to_measure, new_measures = get_roi_to_focus_to_measure(pid, wedding,
                                            template_to_pid_to_cond_to_matrices, roi)
                    pid_to_roi_to_wedding_to_event_to_measure[pid][roi][str(wedding)] = get_event_to_measure(focus_to_measure, roi)
                    measures_list += new_measures
                # this takes average across weddings and tr's and focuses
                pid_to_roi_to_measure[pid][roi] = np.nanmean(measures_list)
        # "for this searchlight and this measure, is the measure reliably above zero across subjects
        roi_to_pval = get_roi_to_pvalue(pid_to_roi_to_measure)
        # light pvals
        if not os.path.exists(output_dir + light_id + "_RoiToPval"):
            with open(output_dir + light_id + "_RoiToPval", "wb") as f:
                f.write(orjson.dumps(roi_to_pval))
        # pid wedding event level neural measures
        neural_measures_path = "/scratch/gpfs/rk1593/clustering_output/searchlights_distilled_neural_measures/"
        if not os.path.exists(neural_measures_path + "each_searchlight/" + light_id):
            with open(neural_measures_path + "each_searchlight/" + light_id, "wb") as f:
                f.write(orjson.dumps(pid_to_roi_to_wedding_to_event_to_measure))

        # the way you can know if doing averages over chunk averages works 
        # as the same as aggregating than averaging is knowing
        # whether or not each chunk is the same size

def divide_chunks(l, chunk_size):
    # looping till length l
    for i in range(0, len(l), chunk_size): 
        yield l[i:i + chunk_size]

# event should be of the form x_y where x is the template event and y is the applied-to event
job_id = int(os.environ["SLURM_ARRAY_TASK_ID"])
output_dir = "/scratch/gpfs/rk1593/clustering_output/brainmap_pvals_example/each_searchlight/"
json_path = "/scratch/gpfs/rk1593/clustering_output/jobs_info_dict_manual_jupyter_without_tuples.json"
f = open(json_path,)
jobs_info_dict = json.load(f)
this_job_searchlights = jobs_info_dict["job_id_to_searchlight_subset"][str(job_id)]
num_chunks = 2
chunk_size = math.floor(len(this_job_searchlights) / num_chunks)
chunks_of_searchlights = divide_chunks(this_job_searchlights, chunk_size = chunk_size)
processes_list = []

for index,chunk in enumerate(chunks_of_searchlights):
    new_p = Process(target = process_chunk, args = (chunk, output_dir))
    new_p.start()
    processes_list.append(new_p)

for p in processes_list:
    p.join()